### Objetivo
Se evaluara la similitud entre palabras a partir de la versión protestante de la biblia.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [2]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
from codecs import ignore_errors
import pandas as pd
f = open("pg10.txt", "r")
documents = pd.DataFrame({"document":[]})
for x in f:
    if x[0] not in "0123456789" or len(x.split(' ')) < 2:
        continue
    # print(x.split(' ', 1)[1])
    df2 = pd.DataFrame({"document":[x.split(' ', 1)[1]]})
    documents = documents.append(df2, ignore_index = True)
df = documents

In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 25017


### 1 - Preprocesamiento

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[0]

['in',
 'the',
 'beginning',
 'god',
 'created',
 'the',
 'heavens',
 'and',
 'the',
 'earth']

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [21]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=1000,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [22]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [23]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 25017


In [24]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 3122


### 3 - Entrenar el modelo generador

In [25]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1897250.0
Loss after epoch 1: 1406046.25
Loss after epoch 2: 1324239.25
Loss after epoch 3: 1251913.0
Loss after epoch 4: 1235473.0
Loss after epoch 5: 1220400.5
Loss after epoch 6: 1156007.0
Loss after epoch 7: 1143543.0
Loss after epoch 8: 1135640.0
Loss after epoch 9: 1129627.0
Loss after epoch 10: 1120744.0
Loss after epoch 11: 1117670.0
Loss after epoch 12: 1113296.0
Loss after epoch 13: 1083451.0
Loss after epoch 14: 1057756.0
Loss after epoch 15: 1054300.0
Loss after epoch 16: 1055294.0
Loss after epoch 17: 1053974.0
Loss after epoch 18: 1061320.0
Loss after epoch 19: 1068800.0


(3783978, 6139340)

### 4 - Ensayar

In [26]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["angel"], topn=10)

[('whirlwind', 0.6389119625091553),
 ('visions', 0.6316908001899719),
 ('messenger', 0.5773029923439026),
 ('elisha', 0.5770173072814941),
 ('row', 0.5575631856918335),
 ('samson', 0.5525782704353333),
 ('gehazi', 0.5453565716743469),
 ('hour', 0.5430376529693604),
 ('pharisee', 0.5419018268585205),
 ('philip', 0.5412166118621826)]

In [27]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["angel"], topn=10)

[('commit', 0.03606146574020386),
 ('thyself', 0.019832352176308632),
 ('lest', 0.011996285058557987),
 ('sin', -0.0013654939830303192),
 ("man's", -0.00898822769522667),
 ('wise', -0.016552120447158813),
 ('shed', -0.01953708752989769),
 ('precious', -0.022870268672704697),
 ('own', -0.024061331525444984),
 ('their', -0.02619529329240322)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["heaven"], topn=10)

[('firmament', 0.5945953130722046),
 ('lightnings', 0.5425839424133301),
 ('heavens', 0.5368310809135437),
 ('throne', 0.5239861011505127),
 ('ascend', 0.5195358991622925),
 ('falling', 0.5088451504707336),
 ('descended', 0.5055993795394897),
 ('lights', 0.4988625645637512),
 ('wonder', 0.498734712600708),
 ('carmel', 0.49806803464889526)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["lord"], topn=10)

[('hosts', 0.6238108277320862),
 ('manoah', 0.6138322353363037),
 ('praises', 0.5962761640548706),
 ('alas', 0.592499852180481),
 ('sir', 0.5865200757980347),
 ('god', 0.5809298157691956),
 ('steward', 0.5782966017723083),
 ('almighty', 0.5749210119247437),
 ('exhortation', 0.5742908716201782),
 ('vengeance', 0.5652214884757996)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab